In [1]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [1]:
!pip install networkx==2.5
!pip install --upgrade linkpred


In [2]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
%cd '/content/drive/MyDrive/COSC4087AtulKumar/Patents/'
%ls

/content/drive/.shortcut-targets-by-id/1pl4EodReTtpQiCti-1pno41FebcE2IjB/COSC4087AtulKumar/Patents
 1000.txt   1182.txt   1363.txt   1544.txt   1725.txt   285.txt   466.txt   647.txt   828.txt
 1001.txt   1183.txt   1364.txt   1545.txt   1726.txt   286.txt   467.txt   648.txt   829.txt
 1002.txt   1184.txt   1365.txt   1546.txt   1727.txt   287.txt   468.txt   649.txt   82.txt
 1003.txt   1185.txt   1366.txt   1547.txt   1728.txt   288.txt   469.txt   64.txt    830.txt
 1004.txt   1186.txt   1367.txt   1548.txt   1729.txt   289.txt   46.txt    650.txt   831.txt
 1005.txt   1187.txt   1368.txt   1549.txt   172.txt    28.txt    470.txt   651.txt   832.txt
 1006.txt   1188.txt   1369.txt   154.txt    1730.txt   290.txt   471.txt   652.txt   833.txt
 1007.txt   1189.txt   136.txt    1550.txt   1731.txt   291.txt   472.txt   653.txt   834.txt
 1008.txt   118.txt    1370.txt   1551.txt   1732.txt   292.txt   473.txt   654.txt   835.txt
 1009.txt   1190.txt   1371.txt   1552.txt   1733.txt   

In [7]:
import re

def remove_numbers_and_words_with_numbers(input_string):
    # Remove words containing numbers
    words = input_string.split()
    filtered_words = [word for word in words if not any(char.isdigit() for char in word)]

    # Remove standalone numbers
    filtered_words = [word for word in filtered_words if not word.isdigit()]

    # Join the filtered words into a new string
    result_string = ' '.join(filtered_words)

    return result_string



In [8]:
def removePunctuation(txt):
    no_punt = "".join([c for c in txt if not c in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'])
    return no_punt


In [9]:
import re

def clean_data(txt):
    # Remove links
    txt = re.sub(r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', ' ', txt)

    # Remove characters that are not letters or numbers
    txt = re.sub(r'[^a-zA-Z0-9\s]', ' ', txt)

    # Remove extra whitespaces
    txt = ' '.join(txt.split())

    return txt


In [10]:
#TOKENIZATION
import re
def tokenize(txt):
  tokens = re.split('\W+', txt)
  return tokens


In [11]:
#REMOVE STOPWORDS
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')

def remove_stopwords(txt):
  new_txt = [word for word in txt if word not in stopwords]
  return new_txt


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [12]:
#LEMMATIZATION
import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')

wn = nltk.WordNetLemmatizer()
def lemmatization(txt):
  text = [wn.lemmatize(word) for word in txt]
  return text


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


#Reading Data

In [15]:
# first year
import string
import pandas as pd

first_period_df = pd.DataFrame(columns=['Title', 'Abstract', 'Claims'])

for ii in range(1, 601):
  filename = str(ii)+".txt"
  with open(filename, 'r') as file:
      my_string = file.read()
      words = my_string.split()
      status = 0
      title =  ""
      abstract = ""
      claims = ""
      for i,word in enumerate(words):
        if(word=="DOCUMENT" and words[i+1] =="ID" ):
          break
        if "DOCUMENT" in word:
          break
        title += word +" "

      # Print each word
      for word in words:
        if(word=="Abstract"):
          status=1
          continue
        if(word=="Claims"):
          status = 2
          continue
        if "Abstract" in word:
          status = 1
          if word[0:8]=="Abstract":
            remaining_char = word.replace("Abstract", "")
            abstract+=remaining_char+" "
          continue
        if "Claims" in word:
          status = 2
          if word[0:6]=="Claims":
            remaining_char = word.replace("Claims", "")
            claims+=remaining_char+" "
          continue

        if(status ==1):
          abstract += word+" "

        if(status ==2):
          claims +=word+" "

      output_title = remove_numbers_and_words_with_numbers(title)
      output_title = output_title.strip()
      output_title = removePunctuation(output_title)
      output_title = clean_data(output_title)
      output_title = ''.join(char for char in output_title if ord(char) < 128)
      output_title =tokenize(output_title.lower())
      output_title = remove_stopwords(output_title)
      output_title = lemmatization(output_title)
      output_title = list(filter(lambda word: word.lower() != "claim".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "according".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "wherein".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "therein".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "canceled".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != " Therefor".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "Thereof".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "Herein".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "Therewith".lower(), output_title))



      output_abs = remove_numbers_and_words_with_numbers(abstract)
      output_abs = output_abs.strip()
      output_abs = removePunctuation(output_abs)
      output_abs = clean_data(output_abs)
      output_abs = ''.join(char for char in output_abs if ord(char) < 128)
      output_abs =tokenize(output_abs.lower())
      output_abs = remove_stopwords(output_abs)
      output_abs = lemmatization(output_abs)
      output_abs = list(filter(lambda word: word.lower() != "claim".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "according".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "wherein".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "therein".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "canceled".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != " Therefor".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "Thereof".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "Herein".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "Therewith".lower(), output_abs))

      output_claim = remove_numbers_and_words_with_numbers(claims)
      output_claim = output_claim.strip()
      output_claim = removePunctuation(output_claim)
      output_claim = clean_data(output_claim)
      output_claim = ''.join(char for char in output_claim if ord(char) < 128)
      output_claim =tokenize(output_claim.lower())
      output_claim = remove_stopwords(output_claim)
      output_claim = lemmatization(output_claim)
      output_claim = list(filter(lambda word: word.lower() != "claim".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "according".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "wherein".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "therein".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "canceled".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != " Therefor".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "Thereof".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "Herein".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "Therewith".lower(), output_claim))

      string_title =""
      for i in output_title:
        string_title += i+" "


      string_abs =""
      for i in output_abs:
        string_abs += i+" "


      string_claim =""
      for i in output_claim:
        string_claim += i+" "
      # print(string)
      first_period_df = first_period_df.append({'Title': string_title.strip(), 'Abstract': string_abs.strip(), 'Claims': string_claim.strip()}, ignore_index=True)


first_period_df

<ipython-input-15-9f5ecc9b49db>:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  first_period_df = first_period_df.append({'Title': string_title.strip(), 'Abstract': string_abs.strip(), 'Claims': string_claim.strip()}, ignore_index=True)
<ipython-input-15-9f5ecc9b49db>:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  first_period_df = first_period_df.append({'Title': string_title.strip(), 'Abstract': string_abs.strip(), 'Claims': string_claim.strip()}, ignore_index=True)
<ipython-input-15-9f5ecc9b49db>:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  first_period_df = first_period_df.append({'Title': string_title.strip(), 'Abstract': string_abs.strip(), 'Claims': string_claim.strip()}, ignore_index=True)
<ipython-inpu

,Title,Abstract,Claims
0,device system method mixing,device system includes mixer comprising threed...,mixing system forming fibrin foam bubble diame...
1,extracellular matrix used scaffold living cell,present invention discloses extracellular matr...,material comprising modified agarose least par...
2,emu oil combination active ingredient treating...,composition comprising emu oil vehicle deliver...,composition comprising emu oil paraben composi...
3,device method treating craniofacial pain,implementation provide method treating craniof...,method treating craniofacial pain patient meth...
4,system method aerosolizing vaporizable material,smoking device generating releasing smoking va...,vaporizer apparatus comprising cartridge conta...
...,...,...,...
595,drug therapy preventing treating glaucoma,provided drug therapy prevention glaucoma prev...,composition comprising salt solvate prostaglan...
596,general medication disposable system,general medication disposal system provided as...,device use disposing amount substance device c...
597,sensor equipped medicinal container,medicinal container bag presented bag include ...,sensor equipped container comprising main body...
598,pharmaceutical composition combination therapy,invention relates use pharmaceutical compositi...,method inducing hypophagia human comprising ad...


In [16]:
# second year
import string
import pandas as pd

second_period_df = pd.DataFrame(columns=['Title', 'Abstract', 'Claims'])

for ii in range(601,1201):
  filename = str(ii)+".txt"
  with open(filename, 'r') as file:
      my_string = file.read()
      words = my_string.split()
      status = 0
      title =  ""
      abstract = ""
      claims = ""
      for i,word in enumerate(words):
        if(word=="DOCUMENT" and words[i+1] =="ID" ):
          break
        if "DOCUMENT" in word:
          break
        title += word +" "

      # Print each word
      for word in words:
        if(word=="Abstract"):
          status=1
          continue
        if(word=="Claims"):
          status = 2
          continue
        if "Abstract" in word:
          status = 1
          if word[0:8]=="Abstract":
            remaining_char = word.replace("Abstract", "")
            abstract+=remaining_char+" "
          continue
        if "Claims" in word:
          status = 2
          if word[0:6]=="Claims":
            remaining_char = word.replace("Claims", "")
            claims+=remaining_char+" "
          continue

        if(status ==1):
          abstract += word+" "

        if(status ==2):
          claims +=word+" "

      output_title = remove_numbers_and_words_with_numbers(title)
      output_title = output_title.strip()
      output_title = removePunctuation(output_title)
      output_title = clean_data(output_title)
      output_title = ''.join(char for char in output_title if ord(char) < 128)
      output_title =tokenize(output_title.lower())
      output_title = remove_stopwords(output_title)
      output_title = lemmatization(output_title)
      output_title = list(filter(lambda word: word.lower() != "claim".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "according".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "wherein".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "therein".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "canceled".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != " Therefor".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "Thereof".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "Herein".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "Therewith".lower(), output_title))



      output_abs = remove_numbers_and_words_with_numbers(abstract)
      output_abs = output_abs.strip()
      output_abs = removePunctuation(output_abs)
      output_abs = clean_data(output_abs)
      output_abs = ''.join(char for char in output_abs if ord(char) < 128)
      output_abs =tokenize(output_abs.lower())
      output_abs = remove_stopwords(output_abs)
      output_abs = lemmatization(output_abs)
      output_abs = list(filter(lambda word: word.lower() != "claim".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "according".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "wherein".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "therein".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "canceled".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != " Therefor".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "Thereof".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "Herein".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "Therewith".lower(), output_abs))

      output_claim = remove_numbers_and_words_with_numbers(claims)
      output_claim = output_claim.strip()
      output_claim = removePunctuation(output_claim)
      output_claim = clean_data(output_claim)
      output_claim = ''.join(char for char in output_claim if ord(char) < 128)
      output_claim =tokenize(output_claim.lower())
      output_claim = remove_stopwords(output_claim)
      output_claim = lemmatization(output_claim)
      output_claim = list(filter(lambda word: word.lower() != "claim".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "according".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "wherein".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "therein".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "canceled".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != " Therefor".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "Thereof".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "Herein".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "Therewith".lower(), output_claim))

      string_title =""
      for i in output_title:
        string_title += i+" "


      string_abs =""
      for i in output_abs:
        string_abs += i+" "


      string_claim =""
      for i in output_claim:
        string_claim += i+" "
      # print(string)
      second_period_df = second_period_df.append({'Title': string_title.strip(), 'Abstract': string_abs.strip(), 'Claims': string_claim.strip()}, ignore_index=True)


second_period_df

<ipython-input-16-e729d6ef8cc2>:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  second_period_df = second_period_df.append({'Title': string_title.strip(), 'Abstract': string_abs.strip(), 'Claims': string_claim.strip()}, ignore_index=True)
<ipython-input-16-e729d6ef8cc2>:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  second_period_df = second_period_df.append({'Title': string_title.strip(), 'Abstract': string_abs.strip(), 'Claims': string_claim.strip()}, ignore_index=True)
<ipython-input-16-e729d6ef8cc2>:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  second_period_df = second_period_df.append({'Title': string_title.strip(), 'Abstract': string_abs.strip(), 'Claims': string_claim.strip()}, ignore_index=True)
<ipytho

,Title,Abstract,Claims
0,compound treatment obesity method use,pentacyclic triterpene weight loss agent provi...,pharmaceutical formulation comprising compound...
1,patient support apparatus communication system,patient support apparatus bed cot stretcher re...,patient support apparatus comprising base fram...
2,negative electrode power storage device power ...,decomposition reaction electrolyte solution li...,power storage device comprising positive elect...
3,medical image processing apparatus,medical image processing apparatus provided ge...,medical image processing apparatus comprising ...
4,apparatus method using ultraviolet light pulsa...,illumination device comprises housing power su...,device treating patient device comprising puls...
...,...,...,...
595,system method insulin pump medical device incl...,medical system includes input assembly receivi...,medical system comprising first medical device...
596,energy harvesting mechanism medical device,embodiment invention provide apparatus system ...,energy harvesting mechanism pacemaker mechanis...
597,internally pressurized medical device,provided pressurized surgical instrument use p...,method maintaining pressure within surgical in...
598,cartridge diagnostic testing,present invention relates cartridge conducting...,method performing diagnostic assay cartridge d...


In [17]:
# third year
import string
import pandas as pd

third_period_df = pd.DataFrame(columns=['Title', 'Abstract', 'Claims'])

for ii in range(1201, 1801):
  filename = str(ii)+".txt"
  with open(filename, 'r') as file:
      my_string = file.read()
      words = my_string.split()
      status = 0
      title =  ""
      abstract = ""
      claims = ""
      for i,word in enumerate(words):
        if(word=="DOCUMENT" and words[i+1] =="ID" ):
          break
        if "DOCUMENT" in word:
          break
        title += word +" "

      # Print each word
      for word in words:
        if(word=="Abstract"):
          status=1
          continue
        if(word=="Claims"):
          status = 2
          continue
        if "Abstract" in word:
          status = 1
          if word[0:8]=="Abstract":
            remaining_char = word.replace("Abstract", "")
            abstract+=remaining_char+" "
          continue
        if "Claims" in word:
          status = 2
          if word[0:6]=="Claims":
            remaining_char = word.replace("Claims", "")
            claims+=remaining_char+" "
          continue

        if(status ==1):
          abstract += word+" "

        if(status ==2):
          claims +=word+" "

      output_title = remove_numbers_and_words_with_numbers(title)
      output_title = output_title.strip()
      output_title = removePunctuation(output_title)
      output_title = clean_data(output_title)
      output_title = ''.join(char for char in output_title if ord(char) < 128)
      output_title =tokenize(output_title.lower())
      output_title = remove_stopwords(output_title)
      output_title = lemmatization(output_title)
      output_title = list(filter(lambda word: word.lower() != "claim".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "according".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "wherein".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "therein".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "canceled".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != " Therefor".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "Thereof".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "Herein".lower(), output_title))
      output_title = list(filter(lambda word: word.lower() != "Therewith".lower(), output_title))



      output_abs = remove_numbers_and_words_with_numbers(abstract)
      output_abs = output_abs.strip()
      output_abs = removePunctuation(output_abs)
      output_abs = clean_data(output_abs)
      output_abs = ''.join(char for char in output_abs if ord(char) < 128)
      output_abs =tokenize(output_abs.lower())
      output_abs = remove_stopwords(output_abs)
      output_abs = lemmatization(output_abs)
      output_abs = list(filter(lambda word: word.lower() != "claim".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "according".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "wherein".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "therein".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "canceled".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != " Therefor".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "Thereof".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "Herein".lower(), output_abs))
      output_abs = list(filter(lambda word: word.lower() != "Therewith".lower(), output_abs))

      output_claim = remove_numbers_and_words_with_numbers(claims)
      output_claim = output_claim.strip()
      output_claim = removePunctuation(output_claim)
      output_claim = clean_data(output_claim)
      output_claim = ''.join(char for char in output_claim if ord(char) < 128)
      output_claim =tokenize(output_claim.lower())
      output_claim = remove_stopwords(output_claim)
      output_claim = lemmatization(output_claim)
      output_claim = list(filter(lambda word: word.lower() != "claim".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "according".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "wherein".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "therein".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "canceled".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != " Therefor".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "Thereof".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "Herein".lower(), output_claim))
      output_claim = list(filter(lambda word: word.lower() != "Therewith".lower(), output_claim))

      string_title =""
      for i in output_title:
        string_title += i+" "


      string_abs =""
      for i in output_abs:
        string_abs += i+" "


      string_claim =""
      for i in output_claim:
        string_claim += i+" "
      # print(string)
      third_period_df = third_period_df.append({'Title': string_title.strip(), 'Abstract': string_abs.strip(), 'Claims': string_claim.strip()}, ignore_index=True)


third_period_df

<ipython-input-17-876d8802134a>:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  third_period_df = third_period_df.append({'Title': string_title.strip(), 'Abstract': string_abs.strip(), 'Claims': string_claim.strip()}, ignore_index=True)
<ipython-input-17-876d8802134a>:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  third_period_df = third_period_df.append({'Title': string_title.strip(), 'Abstract': string_abs.strip(), 'Claims': string_claim.strip()}, ignore_index=True)
<ipython-input-17-876d8802134a>:120: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  third_period_df = third_period_df.append({'Title': string_title.strip(), 'Abstract': string_abs.strip(), 'Claims': string_claim.strip()}, ignore_index=True)
<ipython-inpu

,Title,Abstract,Claims
0,system method optogenetic therapy,configuration described utilizing lightactivat...,method preventing hypertension comprising impl...
1,watersoluble peptide fluorescence material,watersoluble peptide fluorescence material str...,method preparing watersoluble peptide fluoresc...
2,dual specificity antibody fusion,present invention provides dual specificity an...,albumin binding antibody comprising sequence g...
3,treatment diagnosis colon cancer,present invention discloses novel agent method...,method treating colon cancer subject need comp...
4,composition method treating preventing staphyl...,antibody fab region specifically bind staphylo...,method displacing human igg immunoglobulin bou...
...,...,...,...
595,stent antimigration device,stent disclosed elongated body composed bioabs...,stent comprising elongated body composed biode...
596,closed system elastomeric pumping mechanism,medicinal infusion system includes safety valv...,medicinal pumping device comprising medicinal ...
597,decontamination device pharmaceutical vessel,present invention relates embodiment system de...,device decontaminating surface least one vesse...
598,steerable ultrasound catheter,catheter includes catheter body cb cb proximal...,catheter comprising catheter body cb cb proxim...


In [4]:
%cd '/content/drive/MyDrive/COSC4087AtulKumar/Generated Files/'
%ls

/content/drive/.shortcut-targets-by-id/1pl4EodReTtpQiCti-1pno41FebcE2IjB/COSC4087AtulKumar/Generated Files
179700_bigrams.csv          t1_top600.csv                      W_E_LinkPred.csv
300_W5_T1.word2vec_model    t2.csv                             W_LinkPred_i3_ExistingLinks.csv
300_W5_T2.word2vec_model    t2_top600.csv                      W_LinkPred_i3_NonExistingLinks.csv
300_W5_T3.word2vec_model    t3.csv                             W_NE_LinkPred.csv
co_occurrence_matrix_1.csv  t3_top600.csv                      word2vec_D300_W5_T1.npy
co_occurrence_matrix_2.csv  Term_Co_Occurence_Interval_1.gexf  word2vec_D300_W5_T2.npy
co_occurrence_matrix_3.csv  Term_Co_Occurence_Interval_2.gexf  word2vec_D300_W5_T3.npy
t1.csv                      Term_Co_Occurence_Interval_3.gexf


In [19]:
first_period_df.to_csv('t1.csv')
second_period_df.to_csv('t2.csv')
third_period_df.to_csv('t3.csv')

In [20]:
import pandas as pd
first_period_df = pd.read_csv('t1.csv')
second_period_df = pd.read_csv('t2.csv')
third_period_df = pd.read_csv('t3.csv')

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
combined_df = pd.concat([first_period_df, second_period_df, third_period_df], ignore_index=True)
combined_df.dropna(subset=['Title', 'Abstract', 'Claims'], inplace=True)
# df['Combined_Text'] =
texts_combined = combined_df['Title'] + " " + combined_df['Abstract'] + " " + combined_df['Claims']
tfidf_vectorizer = TfidfVectorizer(max_features=3000)
tfidf_matrix = tfidf_vectorizer.fit_transform(texts_combined)
top_3000_unigrams = [unigram for unigram, _ in sorted(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_matrix.sum(axis=0).A1), key=lambda x: x[1], reverse=True)][:3000]
common_unigrams = []
for target_word in top_3000_unigrams:
  if any(first_period_df.apply(lambda row: any(target_word in str(cell) for cell in row), axis=1)) and any(second_period_df.apply(lambda row: any(target_word in str(cell) for cell in row), axis=1)) and any(third_period_df.apply(lambda row: any(target_word in str(cell) for cell in row), axis=1)):
    common_unigrams.append(target_word)
    if(len(common_unigrams)==600):
      break

In [22]:
len(common_unigrams)

600

In [23]:
top_unigrams_with_scores = sorted(zip(tfidf_vectorizer.get_feature_names_out(), tfidf_matrix.sum(axis=0).A1), key=lambda x: x[1], reverse=True)[:200]
for word, score in top_unigrams_with_scores:
    print(f"Word: {word}, TF-IDF: {score}")

Word: device, TF-IDF: 86.39271589718811
Word: first, TF-IDF: 77.56572801415611
Word: method, TF-IDF: 74.03530578322176
Word: said, TF-IDF: 70.1332861059909
Word: second, TF-IDF: 67.06951776270026
Word: system, TF-IDF: 54.678732210967325
Word: one, TF-IDF: 53.64767590050579
Word: portion, TF-IDF: 48.49989670786365
Word: medical, TF-IDF: 46.980074104844476
Word: least, TF-IDF: 46.772930703546116
Word: member, TF-IDF: 42.84129206372682
Word: configured, TF-IDF: 41.51674298847823
Word: end, TF-IDF: 41.30951090378688
Word: comprising, TF-IDF: 41.01830412510941
Word: image, TF-IDF: 37.796498299245215
Word: patient, TF-IDF: 37.35738943163947
Word: comprises, TF-IDF: 36.21513590604279
Word: sensor, TF-IDF: 36.18481368415089
Word: signal, TF-IDF: 35.839401761786014
Word: plurality, TF-IDF: 32.942037751285376
Word: composition, TF-IDF: 31.85647514903915
Word: surface, TF-IDF: 31.769162741307674
Word: light, TF-IDF: 30.987582430149914
Word: distal, TF-IDF: 30.24105557705906
Word: fluid, TF-IDF: 3

In [24]:
import numpy as np
co_occurrence_matrix_first = np.zeros((len(common_unigrams), len(common_unigrams)))

def calculate_distance(word1, word2, word_list):
    try:
        index1 = word_list.index(word1)
        index2 = word_list.index(word2)
        return abs(index1 - index2)
    except ValueError:
        return -1

ii=0
for _, row in first_period_df.iterrows():
    # if ii==1:
    #   break
    combined_text = ' '.join([str(row['Title']), str(row['Abstract']), str(row['Claims'])])
    tokens = combined_text.split()
    # print(tokens)

    for i, unigram1 in enumerate(common_unigrams):
      for j, unigram2 in enumerate(common_unigrams):
        # print(1)
        if(unigram1!=unigram2):

          distance = calculate_distance(unigram1, unigram2, tokens)

          if(distance<4 and distance>0):
            co_occurrence_matrix_first[i][j] += 1
            # print(f"Distance between '{unigram1}' and '{unigram2}' in a1: {distance}")



    # print("done")
    ii+=1

df_co_occurrence_matrix_first = pd.DataFrame(co_occurrence_matrix_first, index=common_unigrams, columns=common_unigrams)
df_co_occurrence_matrix_first.to_csv('co_occurrence_matrix_1.csv')

In [25]:
import numpy as np
co_occurrence_matrix_second = np.zeros((len(common_unigrams), len(common_unigrams)))

def calculate_distance(word1, word2, word_list):
    try:
        index1 = word_list.index(word1)
        index2 = word_list.index(word2)
        return abs(index1 - index2)
    except ValueError:
        return -1

ii=0
for _, row in second_period_df.iterrows():
    # if ii==1:
    #   break
    combined_text = ' '.join([str(row['Title']), str(row['Abstract']), str(row['Claims'])])
    tokens = combined_text.split()
    # print(tokens)

    for i, unigram1 in enumerate(common_unigrams):
      for j, unigram2 in enumerate(common_unigrams):
        # print(1)
        if(unigram1!=unigram2):

          distance = calculate_distance(unigram1, unigram2, tokens)

          if(distance<4 and distance>0):
            co_occurrence_matrix_second[i][j] += 1
            # print(f"Distance between '{unigram1}' and '{unigram2}' in a1: {distance}")



    # print("done")
    ii+=1

df_co_occurrence_matrix_second = pd.DataFrame(co_occurrence_matrix_second, index=common_unigrams, columns=common_unigrams)
df_co_occurrence_matrix_second.to_csv('co_occurrence_matrix_2.csv')

In [26]:
import numpy as np
co_occurrence_matrix_third = np.zeros((len(common_unigrams), len(common_unigrams)))

def calculate_distance(word1, word2, word_list):
    try:
        index1 = word_list.index(word1)
        index2 = word_list.index(word2)
        return abs(index1 - index2)
    except ValueError:
        return -1

ii=0
for _, row in third_period_df.iterrows():
    # if ii==1:
    #   break
    combined_text = ' '.join([str(row['Title']), str(row['Abstract']), str(row['Claims'])])
    tokens = combined_text.split()
    # print(tokens)

    for i, unigram1 in enumerate(common_unigrams):
      for j, unigram2 in enumerate(common_unigrams):
        # print(1)
        if(unigram1!=unigram2):

          distance = calculate_distance(unigram1, unigram2, tokens)

          if(distance<4 and distance>0):
            co_occurrence_matrix_third[i][j] += 1
            # print(f"Distance between '{unigram1}' and '{unigram2}' in a1: {distance}")



    # print("done")
    ii+=1

df_co_occurrence_matrix_third = pd.DataFrame(co_occurrence_matrix_third, index=common_unigrams, columns=common_unigrams)
df_co_occurrence_matrix_third.to_csv('co_occurrence_matrix_3.csv')

In [32]:
import nltk
nltk.download('punkt')
import numpy as np
import pandas as pd

file = "co_occurrence_matrix_1.csv"

df = pd.read_csv(file)
unigrams = df.iloc[:,[0]].values.tolist()
i = 0
number_bigrams = 0
bigrams = []
for unigram1 in unigrams:
    l=0
    for unigram2 in unigrams[(i+1):]:
        l+=1
        if str(unigram2)=="[nan]":
            unigram2 = "null"
            unigram1 = "".join(unigram1)
        elif str(unigram1)=="[nan]":
            unigram1 = "null"
            unigram2 = "".join(unigram2)
        else:
            unigram1 = "".join(unigram1)
            unigram2 = "".join(unigram2)
        if unigram1 != unigram2:
            bigrams.append(unigram1 + " "+ unigram2)
            number_bigrams += 1
    i += 1

print("\nIn total %i bigrams generated!\n" %(number_bigrams))

pd.DataFrame(bigrams).to_csv("%i_bigrams.csv" %number_bigrams, sep=";", header=["Bigram"])


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.



In total 179700 bigrams generated!



In [33]:
from gensim.models import Word2Vec
anzahl_terme = 1000
window_size_bigrams = 4
Start = 1
End = 3
dimensions= 300
window_size=5
min_count=1
noise_terms=15
len_texts = []
for time_slice in range(Start, End+1):

    """Load in data"""
    if time_slice==1:
        csv_file = "t1.csv"
    if time_slice==2:
        csv_file = "t2.csv"
    if time_slice==3:
        csv_file = "t3.csv"

    """Read csv file"""
    df = pd.read_csv(csv_file).dropna()
    texts_raw = df["Title"] + " "+ df["Abstract"] +" " + df["Claims"]
    texts=[''.join([i for i in str(s) if not i.isdigit()]) for s in texts_raw]
    texts=[i.replace("."," ") for i in texts]
    texts=[i.replace(","," ") for i in texts]
    texts=[i.replace("!"," ") for i in texts]
    texts=[i.replace("?"," ") for i in texts]
    texts=[i.replace("&"," ") for i in texts]
    texts=[i.replace("-"," ") for i in texts]
    texts=[i.replace(";"," ") for i in texts]
    texts=[i.replace("/"," ") for i in texts]
    texts=[i.replace(")"," ") for i in texts]
    texts=[i.replace("("," ") for i in texts]
    texts=[i.replace("+"," ") for i in texts]
    texts=[i.replace("="," ") for i in texts]
    texts=[i.replace("\\"," ") for i in texts]
    texts=[i.replace(":"," ") for i in texts]
    texts=[i.replace("'"," ") for i in texts]
    texts=[i.replace("`"," ") for i in texts]
    texts=[i.replace("´"," ") for i in texts]
    texts =[i.lower() for i in texts]
    len_texts.append(len(texts))
    print("\nNumber of patents in timeslice %i: %i\n" %(time_slice, len(texts)))

    del texts_raw
    """Stoplist + Formatting"""
    stoplist = ". , ! 1. 2. 3. 4. 5. 6. 7. 8. 9. 0. 1 2 3 4 5 6 7 8 9 0"
    stoplist = set(stoplist.split())
    texts=[[word.lower() for word in document.split() if not (word in stoplist or len(word)<3)] for document in texts]

    #Training word2vec
    dimensions = dimensions
    window_size = window_size
    word2vec_model = Word2Vec(sentences=texts, vector_size=dimensions, window=window_size, epochs=20, min_count=min_count, workers=1, sg=1, seed=0, hs=1, negative=noise_terms)

    word2vec_model.save("%i_W%i_T%i.word2vec_model" %(dimensions,window_size,time_slice))

    dictionary = {}
    similarity_value= []
    for h in bigrams:
        h = h.split(" ")
        h.sort()
        first_word, second_word = h[0], h[1]

        try:
             value = round(float(word2vec_model.wv.similarity(first_word, second_word)), 6)
             similarity_value.append(value)
             dictionary[str(first_word), str(second_word)] = value
        except KeyError:
            value = 0
            similarity_value.append(value)
            dictionary[str(first_word), str(second_word)] = value

    df = pd.DataFrame(similarity_value, index = bigrams)
    np.save("word2vec_D300_W5_T%i.npy" %time_slice, dictionary)



Number of patents in timeslice 1: 586




Number of patents in timeslice 2: 595


Number of patents in timeslice 3: 596



In [34]:
import pandas as pd

window_size_bigrams = 4

# adjacency matrix
co_occurance_t1 = pd.read_csv("co_occurrence_matrix_1.csv")
co_occurance_t2 = pd.read_csv("co_occurrence_matrix_2.csv")
co_occurance_t3 = pd.read_csv("co_occurrence_matrix_3.csv")

def clean_Intervall_Lists(table):
    table = table.set_index("Unnamed: 0", drop = True)
    return table

clean_co_occurance_t1 = clean_Intervall_Lists(co_occurance_t1)
clean_co_occurance_t2 = clean_Intervall_Lists(co_occurance_t2)
clean_co_occurance_t3 = clean_Intervall_Lists(co_occurance_t3)

term_list = list(co_occurance_t1["Unnamed: 0"])
def sort_Tuples(input_tuple):
    tuple_list = list(input_tuple)
    tuple_list.sort()
    return tuple(tuple_list)

from itertools import combinations

combination_dict_t1 = {}
combination_dict_t2 = {}
combination_dict_t3 = {}

for combination_dict, co_occurance_matrix in zip([combination_dict_t1,combination_dict_t2,combination_dict_t3],[clean_co_occurance_t1,clean_co_occurance_t2,clean_co_occurance_t3]):
    for combination in combinations(term_list,2):

        new_combination = sort_Tuples(combination)
        value = co_occurance_matrix.loc[new_combination[0],new_combination[1]]
        if str(value) == "0.0":
            continue
        else:
            combination_dict[new_combination] = value
import networkx as nx
for i, combination_dict in zip([1,2,3],[combination_dict_t1,combination_dict_t2,combination_dict_t3]):
    G = nx.Graph()
    for key in combination_dict.keys():
        G.add_edge(key[0], key[1], weight = combination_dict[key])
    save_file = "Term_Co_Occurence_Interval_" + str(i) + ".gexf"
    nx.write_gexf(G,save_file)


In [35]:
# For T1
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

first_period_df['Combined_Text'] = first_period_df['Title'].astype(str) + ' ' + first_period_df['Abstract'].astype(str) + ' ' + first_period_df['Claims'].astype(str)
unigrams_str = ' '.join(common_unigrams)

#TF-IDF
tfidf_vectorizer = TfidfVectorizer(vocabulary=common_unigrams, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(first_period_df['Combined_Text'])

unigram_tfidf_values = {}
for unigram in common_unigrams:
    unigram_index = tfidf_vectorizer.vocabulary_.get(unigram)
    if unigram_index is not None:
        tfidf_value = tfidf_matrix[:, unigram_index].sum()
        unigram_tfidf_values[unigram] = tfidf_value

tfidf_list = []
for i in common_unigrams:
  tfidf_list.append(unigram_tfidf_values[i])

data = {'Term': [], 'tf': []}
for specific_word in common_unigrams:
  count = first_period_df['Title'].str.count(specific_word).sum()
  count+= first_period_df['Abstract'].str.count(specific_word).sum()
  count+= first_period_df['Claims'].str.count(specific_word).sum()
  data['Term'].append(specific_word)
  data['tf'].append(count)

df_word_counts = pd.DataFrame(data)
df_word_counts['tf-idf*'] = tfidf_list
df_word_counts.to_csv('t1_top600.csv', sep =',')
df_word_counts

,Term,tf,tf-idf*
0,device,5776.0,38.459825
1,first,5778.0,0.000000
2,method,5805.0,40.440705
3,said,2774.0,28.406263
4,second,5365.0,31.047159
...,...,...,...
595,recess,73.0,1.263343
596,parallel,104.0,1.498644
597,gene,1677.0,3.208012
598,handheld,35.0,0.910222


In [36]:
#FOR T2
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

second_period_df['Combined_Text'] = second_period_df['Title'].astype(str) + ' ' + second_period_df['Abstract'].astype(str) + ' ' + second_period_df['Claims'].astype(str)
unigrams_str = ' '.join(common_unigrams)

# TF-IDF
tfidf_vectorizer = TfidfVectorizer(vocabulary=common_unigrams, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(second_period_df['Combined_Text'])

unigram_tfidf_values = {}
for unigram in common_unigrams:
    unigram_index = tfidf_vectorizer.vocabulary_.get(unigram)
    if unigram_index is not None:
        tfidf_value = tfidf_matrix[:, unigram_index].sum()
        unigram_tfidf_values[unigram] = tfidf_value


tfidf_list = []
for i in common_unigrams:
  tfidf_list.append(unigram_tfidf_values[i])
tfidf_list


data = {'Term': [], 'tf': []}
for specific_word in common_unigrams:
  count = second_period_df['Title'].str.count(specific_word).sum()
  count+= second_period_df['Abstract'].str.count(specific_word).sum()
  count+= second_period_df['Claims'].str.count(specific_word).sum()
  data['Term'].append(specific_word)
  data['tf'].append(count)

df_word_counts = pd.DataFrame(data)
df_word_counts['tf-idf*'] = tfidf_list
df_word_counts.to_csv('t2_top600.csv', sep =',')
df_word_counts

,Term,tf,tf-idf*
0,device,6967.0,42.248081
1,first,6380.0,0.000000
2,method,5269.0,36.623811
3,said,4459.0,35.106760
4,second,5622.0,30.181035
...,...,...,...
595,recess,148.0,2.013628
596,parallel,149.0,1.802320
597,gene,1357.0,0.572294
598,handheld,121.0,2.066234


In [37]:
#FOR T3
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

third_period_df['Combined_Text'] = third_period_df['Title'].astype(str) + ' ' + third_period_df['Abstract'].astype(str) + ' ' + third_period_df['Claims'].astype(str)
unigrams_str = ' '.join(common_unigrams)

# TF-IDF
tfidf_vectorizer = TfidfVectorizer(vocabulary=common_unigrams, stop_words='english')
tfidf_matrix = tfidf_vectorizer.fit_transform(third_period_df['Combined_Text'])

unigram_tfidf_values = {}
for unigram in common_unigrams:
    unigram_index = tfidf_vectorizer.vocabulary_.get(unigram)
    if unigram_index is not None:
        tfidf_value = tfidf_matrix[:, unigram_index].sum()
        unigram_tfidf_values[unigram] = tfidf_value

tfidf_list = []
for i in common_unigrams:
  tfidf_list.append(unigram_tfidf_values[i])
tfidf_list


data = {'Term': [], 'tf': []}
for specific_word in common_unigrams:
  count = third_period_df['Title'].str.count(specific_word).sum()
  count+= third_period_df['Abstract'].str.count(specific_word).sum()
  count+= third_period_df['Claims'].str.count(specific_word).sum()
  data['Term'].append(specific_word)
  data['tf'].append(count)

df_word_counts = pd.DataFrame(data)
df_word_counts['tf-idf*'] = tfidf_list
df_word_counts.to_csv('t3_top600.csv', sep =',')
df_word_counts

,Term,tf,tf-idf*
0,device,6745.0,40.554247
1,first,6933.0,0.000000
2,method,4506.0,29.611146
3,said,3206.0,30.371152
4,second,6142.0,33.310721
...,...,...,...
595,recess,132.0,1.848734
596,parallel,146.0,1.639133
597,gene,1267.0,1.394797
598,handheld,79.0,1.447781


In [4]:
import pandas as pd
import networkx as nx
import linkpred as lp
import numpy as np
from linkpred.preprocess import without_selfloops

num_terms = 600

window_size_bigrams = 4

#read word2vec
word2vec_t1 = np.load("word2vec_D300_W5_T1.npy", allow_pickle = True).item()
word2vec_t2 = np.load("word2vec_D300_W5_T2.npy", allow_pickle = True).item()
word2vec_t3 = np.load("word2vec_D300_W5_T3.npy", allow_pickle = True).item()


#read node tables
nodes_int_1 = pd.read_csv("t1_top600.csv")
nodes_int_2 = pd.read_csv("t2_top600.csv")
nodes_int_3 = pd.read_csv("t3_top600.csv")

#assign term frequencies
Tf_dict_1 = {term:value for term, value in zip(nodes_int_1.Term, nodes_int_1.tf)}
Tf_dict_2 = {term:value for term, value in zip(nodes_int_2.Term, nodes_int_2.tf)}
Tf_dict_3 = {term:value for term, value in zip(nodes_int_3.Term, nodes_int_3.tf)}

#assign tfidf values
Tfidf_dict_1 = {term:value for term, value in zip(nodes_int_1.Term, nodes_int_1["tf-idf*"])}
Tfidf_dict_2 = {term:value for term, value in zip(nodes_int_2.Term, nodes_int_2["tf-idf*"])}
Tfidf_dict_3 =  {term:value for term, value in zip(nodes_int_3.Term, nodes_int_3["tf-idf*"])}

def Tuple_Sort(tupel):
    liste = list(tupel)
    liste.sort()
    return(liste[0],liste[1])

#load and read the all three year network file
file_name = "Term_Co_Occurence_Interval_1.gexf"
G1 = nx.read_gexf(file_name)

file_name = "Term_Co_Occurence_Interval_2.gexf"
G2 = nx.read_gexf(file_name)

file_name = "Term_Co_Occurence_Interval_3.gexf"
G3 = nx.read_gexf(file_name)

#transform networks to undirected
H1 = G1.to_undirected()
H1 = without_selfloops(H1)

H2 = G2.to_undirected()
H2 = without_selfloops(H2)

H3 = G3.to_undirected()
H3 = without_selfloops(H3)

#Set weight
weight = None
name_prefix = "W"

result_list = []
i = 0
interval = 1


# Year 2017 & 2018 - Computing scores for existing edges
for network, nextnetwork, word2vec, tf_dict, tfidf_dict in [(H1,H2,word2vec_t1, Tf_dict_1, Tfidf_dict_1), (H2, H3, word2vec_t2, Tf_dict_2, Tfidf_dict_2)]:
    print("\nCalculating Preferential Attachment")
    pad_results = {}
    pa_preds = nx.preferential_attachment(network, nx.edges(network))
    for u, v, p, in pa_preds:
        edge = (u,v)
        pad_results[Tuple_Sort(edge)] = p

    print("\nCalculating Simrank")
    simrank = lp.predictors.SimRank(network, excluded = nx.non_edges(network))
    simrank_results = simrank.predict(c=0.85, weight = weight)

    print("\nCalculating RootedPageRank")
    rootedpagerank = lp.predictors.RootedPageRank(network, excluded = nx.non_edges(network))
    rpr_results = rootedpagerank.predict(weight = weight)

    print("\nCalculating Katz")
    katz = lp.predictors.Katz(network, excluded = nx.non_edges(network))
    katz_results = katz.predict(weight = weight)

    print("\nCalculating CommonNeighbours")
    commonneighbours = lp.predictors.CommonNeighbours(network, excluded = nx.non_edges(network))
    cn_results = commonneighbours.predict(weight = weight)

    print("\nCalculating AdamicAdar")
    adamicadar = lp.predictors.AdamicAdar(network, excluded = nx.non_edges(network))
    adamicadar_results = adamicadar.predict(weight = weight)

    print("\nCalculating Jaccard")
    jaccard = lp.predictors.Jaccard(network, excluded = nx.non_edges(network))
    jaccard_results = jaccard.predict(weight = weight)

    print("\nCalculating ResourceAllocation")
    resourceallocation = lp.predictors.ResourceAllocation(network, excluded = nx.non_edges(network))
    ra_results = resourceallocation.predict(weight = weight)

    a=0
    for edge in nx.edges(network):
        edge = Tuple_Sort(edge)
        word_similarity = word2vec[edge]
        tf_value_1 = tf_dict[edge[0]]
        tf_value_2 = tf_dict[edge[1]]
        tfidf_value_1 = tfidf_dict[edge[0]]
        tfidf_value_2 = tfidf_dict[edge[1]]
        result_list.append([interval, edge[0], edge[1], tf_value_1, tf_value_2, tfidf_value_1, tfidf_value_2, word_similarity, adamicadar_results[edge], cn_results[edge], jaccard_results[edge], pad_results[edge], ra_results[edge], katz_results[edge], rpr_results[edge], simrank_results[edge], nextnetwork.has_edge(edge[0],edge[1])])
        a += 1


    print("\nInterval %i of 2 of list 1 of 4 completed" %(interval))
    interval += 1

existing_pred_data = pd.DataFrame(result_list, columns = ["NetworkInterval","u","v", "TF_u","TF_v","TFIDF_u","TFIDF_v","Word_Similarity", "AdamicAdar", "CommonNeighbours", "Jaccard", "PreferentialAttachment", "ResourceAllocation", "Katz", "RootedPageRank", "Simrank","Existing_in_T+1"])
existing_pred_data["Existing_in_T+1"] = [int(x) for x in existing_pred_data["Existing_in_T+1"]]

save_name = name_prefix + "_E_LinkPred.csv"
existing_pred_data.to_csv(save_name)
print("\n1 of 4 finished\n")

result_list = []
interval = 1

# Year 2017 & 2018 - Computing scores for Non-existing edges
for network, nextnetwork, word2vec, tf_dict, tfidf_dict in [(H1,H2,word2vec_t1, Tf_dict_1, Tfidf_dict_1), (H2,H3,word2vec_t2,Tf_dict_2,Tfidf_dict_2)]:
    print("\nCalculating Preferential Attachment")
    pad_results = {}
    pa_preds = nx.preferential_attachment(network, nx.non_edges(network))
    for u, v, p, in pa_preds:
        edge = (u,v)
        pad_results[Tuple_Sort(edge)] = p

    print("\nCalculating Simrank")
    simrank = lp.predictors.SimRank(network, excluded = nx.edges(network))
    simrank_results = simrank.predict(c=0.85, weight = weight)

    print("\nCalculating RootedPageRank")
    rootedpagerank = lp.predictors.RootedPageRank(network, excluded = nx.edges(network))
    rpr_results = rootedpagerank.predict(weight = weight)

    print("\nCalculating Katz")
    katz = lp.predictors.Katz(network, excluded = nx.edges(network))
    katz_results = katz.predict(weight = weight)

    print("Calculating CommonNeighbours")
    commonneighbours = lp.predictors.CommonNeighbours(network, excluded = nx.edges(network))
    cn_results = commonneighbours.predict(weight = weight)

    print("\nCalculating AdamicAdar")
    adamicadar = lp.predictors.AdamicAdar(network, excluded = nx.edges(network))
    adamicadar_results = adamicadar.predict(weight = weight)

    print("\nCalculating Jaccard")
    jaccard = lp.predictors.Jaccard(network, excluded = nx.edges(network))
    jaccard_results = jaccard.predict(weight = weight)

    print("\nCalculating ResourceAllocation")
    resourceallocation = lp.predictors.ResourceAllocation(network, excluded = nx.edges(network))
    ra_results = resourceallocation.predict(weight = weight)

    a = 0
    for edge in nx.non_edges(network):
        edge = Tuple_Sort(edge)
        word_similarity = word2vec[edge]
        tf_value_1 = tf_dict[edge[0]]
        tf_value_2 = tf_dict[edge[1]]
        tfidf_value_1 = tfidf_dict[edge[0]]
        tfidf_value_2 = tfidf_dict[edge[1]]
        result_list.append([interval, edge[0], edge[1], tf_value_1, tf_value_2, tfidf_value_1, tfidf_value_2, word_similarity, adamicadar_results[edge], cn_results[edge], jaccard_results[edge], pad_results[edge], ra_results[edge], katz_results[edge], rpr_results[edge], simrank_results[edge], nextnetwork.has_edge(edge[0],edge[1])])
        a += 1

    print("\nInterval %i of 2 of list 2 of 4 completed" %(interval))
    interval += 1

non_existing_pred_data = pd.DataFrame(result_list, columns = ["NetworkInterval","u","v", "TF_u","TF_v","TFIDF_u","TFIDF_v","Word_Similarity", "AdamicAdar", "CommonNeighbours", "Jaccard", "PreferentialAttachment", "ResourceAllocation", "Katz", "RootedPageRank", "Simrank","Existing_in_T+1"])
non_existing_pred_data["Existing_in_T+1"] = [int(x) for x in non_existing_pred_data["Existing_in_T+1"]]

save_name = name_prefix + "_NE_LinkPred.csv"
non_existing_pred_data.to_csv(save_name)
print("\n2 of 4 finished\n")


# Year 2019 - Computing scores for existing edges
interval=3
result_list = []
tf_dict = Tf_dict_3
tfidf_dict = Tfidf_dict_3
word2vec = word2vec_t3
network = H3
print("\nCalculating Simrank")
simrank = lp.predictors.SimRank(network, excluded = nx.non_edges(network))
simrank_results = simrank.predict(c=0.85, weight = weight)

print("\nCalculating RootedPageRank")
rootedpagerank = lp.predictors.RootedPageRank(network, excluded = nx.non_edges(network))
rpr_results = rootedpagerank.predict(weight = weight)

print("\nCalculating Katz")
katz = lp.predictors.Katz(network, excluded = nx.non_edges(network))
katz_results = katz.predict(weight = weight)

print("\nCalculating CommonNeighbours")
commonneighbours = lp.predictors.CommonNeighbours(network, excluded = nx.non_edges(network))
cn_results = commonneighbours.predict(weight = weight)

print("\nCalculating AdamicAdar")
adamicadar = lp.predictors.AdamicAdar(network, excluded = nx.non_edges(network))
adamicadar_results = adamicadar.predict(weight = weight)

print("\nCalculating Jaccard")
jaccard = lp.predictors.Jaccard(network, excluded = nx.non_edges(network))
jaccard_results = jaccard.predict(weight = weight)

print("\nCalculating ResourceAllocation")
resourceallocation = lp.predictors.ResourceAllocation(network, excluded = nx.non_edges(network))
ra_results = resourceallocation.predict(weight = weight)

print("\nCalculating Preferential Attachment")
pad_results = {}
pa_preds = nx.preferential_attachment(network, nx.edges(network))
for u, v, p, in pa_preds:
    edge = (u,v)
    pad_results[Tuple_Sort(edge)] = p

a = 0
for edge in nx.edges(network):
    edge = Tuple_Sort(edge)
    word_similarity = word2vec[edge]
    tf_value_1 = tf_dict[edge[0]]
    tf_value_2 = tf_dict[edge[1]]
    tfidf_value_1 = tfidf_dict[edge[0]]
    tfidf_value_2 = tfidf_dict[edge[1]]
    result_list.append([interval, edge[0], edge[1], tf_value_1, tf_value_2, tfidf_value_1, tfidf_value_2, word_similarity, adamicadar_results[edge], cn_results[edge], jaccard_results[edge], pad_results[edge], ra_results[edge], katz_results[edge], rpr_results[edge], simrank_results[edge]])
    a += 1


existing_i3_pred_data = pd.DataFrame(result_list, columns = ["NetworkInterval","u","v", "TF_u","TF_v","TFIDF_u","TFIDF_v","Word_Similarity", "AdamicAdar", "CommonNeighbours", "Jaccard", "PreferentialAttachment", "ResourceAllocation", "Katz", "RootedPageRank", "Simrank"])

save_name = name_prefix + "_LinkPred_i3_ExistingLinks.csv"
existing_i3_pred_data.to_csv(save_name)
print("\n3 of 4 finished\n")

# Year 2019 - Computing scores for Non-existing edges
i = 0
network = H3
result_list = []
interval=3
print("\nCalculating Simrank")
simrank = lp.predictors.SimRank(network, excluded = nx.edges(network))
simrank_results = simrank.predict(c=0.85, weight = weight)

print("\nCalculating RootedPageRank")
rootedpagerank = lp.predictors.RootedPageRank(network, excluded = nx.edges(network))
rpr_results = rootedpagerank.predict(weight = weight)

print("\nCalculating Katz")
katz = lp.predictors.Katz(network, excluded = nx.non_edges(network))
katz_results = katz.predict(weight = weight)

print("\nCalculating CommonNeighbours")
commonneighbours = lp.predictors.CommonNeighbours(network, excluded = nx.edges(network))
cn_results = commonneighbours.predict(weight = weight)

print("\nCalculating AdamicAdar")
adamicadar = lp.predictors.AdamicAdar(network, excluded = nx.edges(network))
adamicadar_results = adamicadar.predict(weight = weight)

print("\nCalculating Jaccard")
jaccard = lp.predictors.Jaccard(network, excluded = nx.edges(network))
jaccard_results = jaccard.predict(weight = weight)

print("\nCalculating ResourceAllocation")
resourceallocation = lp.predictors.ResourceAllocation(network, excluded = nx.edges(network))
ra_results = resourceallocation.predict(weight = weight)

print("\nCalculating Preferential Attachment")
pad_results = {}
pa_preds = nx.preferential_attachment(network, nx.non_edges(network))
for u, v, p, in pa_preds:
    edge = (u,v)
    pad_results[Tuple_Sort(edge)] = p

a = 0
for edge in nx.non_edges(network):
    edge = Tuple_Sort(edge)
    word_similarity = word2vec[edge]
    tf_value_1 = tf_dict[edge[0]]
    tf_value_2 = tf_dict[edge[1]]
    tfidf_value_1 = tfidf_dict[edge[0]]
    tfidf_value_2 = tfidf_dict[edge[1]]
    result_list.append([interval, edge[0], edge[1], tf_value_1, tf_value_2, tfidf_value_1, tfidf_value_2, word_similarity, adamicadar_results[edge], cn_results[edge], jaccard_results[edge], pad_results[edge], ra_results[edge], katz_results[edge], rpr_results[edge], simrank_results[edge]])
    a += 1


non_existing_i3_pred_data = pd.DataFrame(result_list, columns = ["NetworkInterval","u","v", "TF_u","TF_v","TFIDF_u","TFIDF_v","Word_Similarity", "AdamicAdar", "CommonNeighbours", "Jaccard", "PreferentialAttachment", "ResourceAllocation", "Katz", "RootedPageRank", "Simrank"])

save_name = name_prefix + "_LinkPred_i3_NonExistingLinks.csv"
non_existing_i3_pred_data.to_csv(save_name)



Calculating Preferential Attachment

Calculating Simrank

Calculating RootedPageRank


Calculating Katz


Calculating CommonNeighbours

Calculating AdamicAdar

Calculating Jaccard

Calculating ResourceAllocation

Interval 1 of 2 of list 1 of 4 completed

Calculating Preferential Attachment

Calculating Simrank

Calculating RootedPageRank


Calculating Katz


Calculating CommonNeighbours

Calculating AdamicAdar

Calculating Jaccard

Calculating ResourceAllocation

Interval 2 of 2 of list 1 of 4 completed

1 of 4 finished


Calculating Preferential Attachment

Calculating Simrank

Calculating RootedPageRank


Calculating Katz

Calculating CommonNeighbours

Calculating AdamicAdar

Calculating Jaccard

Calculating ResourceAllocation

Interval 1 of 2 of list 2 of 4 completed

Calculating Preferential Attachment

Calculating Simrank

Calculating RootedPageRank


Calculating Katz

Calculating CommonNeighbours

Calculating AdamicAdar

Calculating Jaccard

Calculating ResourceAllocation

Inter

#ML models

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load file
prefix = "W"
filename = prefix + "_NE_LinkPred.csv"
non_existing_pred_data = pd.read_csv(filename)
filename = prefix + "_E_LinkPred.csv"
existing_pred_data = pd.read_csv(filename)
df1 = pd.DataFrame(non_existing_pred_data.iloc[:,[4,5,6,7,8,9,10,11,12,13,14,15,16,17]].values)
df2 = pd.DataFrame(existing_pred_data.iloc[:,[4,5,6,7,8,9,10,11,12,13,14,15,16,17]].values)
combined_data = pd.concat([df1,df2])
rounded = combined_data.round(4)
data = combined_data.drop_duplicates(keep = "first")
data = rounded.drop_duplicates(keep = "first")

y = data.iloc[:, -1].values
X = data.iloc[:,[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]].values

# Splitting the dataset into the Training set and Test set
test_size = 0.2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_size, random_state = 0, stratify = y)

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)


In [6]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
from keras.layers import Input, Dense, Embedding, LSTM, GlobalMaxPooling1D
from keras.models import Model
import tensorflow as tf
from tensorflow import keras

In [7]:
model_mlp=keras.models.Sequential()
model_mlp.add(keras.layers.Flatten(input_shape=[13,]))
model_mlp.add(keras.layers.Dense(128, activation="relu"))
model_mlp.add(keras.layers.Dense(64, activation="relu"))
model_mlp.add(keras.layers.Dense(32, activation="relu"))
model_mlp.add(keras.layers.Dense(16, activation="relu"))
model_mlp.add(keras.layers.Dense(4, activation="relu"))
model_mlp.add(keras.layers.Dense(1, activation="sigmoid"))

In [8]:
optimizer = Adam(learning_rate=3e-4)
model_mlp.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [9]:
history = model_mlp.fit(X_train, y_train, epochs=2)

Epoch 1/2
8956/8956 [==============================] - 51s 6ms/step - loss: 0.2357 - accuracy: 0.9178
Epoch 2/2
8956/8956 [==============================] - 47s 5ms/step - loss: 0.2251 - accuracy: 0.9201


In [10]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_train_pred = model_mlp.predict(X_train)
for i in range(len(y_train_pred)):
  if (y_train_pred[i]>=0.5):
    y_train_pred[i]=1
  else:
    y_train_pred[i]=0
cm = confusion_matrix(y_train, y_train_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("\nTraining Data Results: ")
print("Accuracy on training data:  ", round(model_mlp.evaluate(X_train, y_train)[1]*100,2), "%")
print("Precision on training data: ", round(precision*100,2), "%")
print("Recall on training data:    ", round(recall*100, 2), "%")
print("F1 score on training data:  ", round(f1*100, 2), "%")
print()
print("Test Data Results: ")
y_pred = model_mlp.predict(X_test)
for i in range(len(y_pred)):
  if (y_pred[i]>=0.5):
    y_pred[i]=1
  else:
    y_pred[i]=0
cm = confusion_matrix(y_test, y_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("Accuracy on test data:  ", round(model_mlp.evaluate(X_test, y_test)[1]*100,2), "%")
print("Precision on test data: ",  round(precision*100, 2), "%")
print("Recall on test data:    ", round(recall*100, 2), "%")
print("F1 score on test data:  ",  round(f1*100, 2), "%")

8956/8956 [==============================] - 32s 4ms/step

Training Data Results: 
8956/8956 [==============================] - 34s 4ms/step - loss: 0.2251 - accuracy: 0.9201
Accuracy on training data:   92.01 %
Precision on training data:  98.67 %
Recall on training data:     92.99 %
F1 score on training data:   95.75 %

Test Data Results: 
2239/2239 [==============================] - 12s 5ms/step - loss: 0.2248 - accuracy: 0.9199
Accuracy on test data:   91.99 %
Precision on test data:  98.67 %
Recall on test data:     92.97 %
F1 score on test data:   95.74 %


In [11]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten

model_lstm = Sequential()

model_lstm.add(LSTM(200, input_shape=(13, 1), dropout=0.2, return_sequences=True))
model_lstm.add(Dense(100, activation="relu"))
model_lstm.add((LSTM(64, dropout=0.2)))
model_lstm.add(Dense(5, activation="relu"))
model_lstm.add(Flatten())
model_lstm.add(Dense(1, activation="sigmoid"))


In [12]:
optimizer = Adam(learning_rate=3e-4)
model_lstm.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [13]:
history = model_lstm.fit(X_train, y_train, epochs=2)

Epoch 1/2
8956/8956 [==============================] - 820s 91ms/step - loss: 0.2358 - accuracy: 0.9163
Epoch 2/2
8956/8956 [==============================] - 837s 93ms/step - loss: 0.2320 - accuracy: 0.9181


In [27]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_train_pred = model_lstm.predict(X_train)
for i in range(len(y_train_pred)):
  if (y_train_pred[i]>=0.5):
    y_train_pred[i]=1
  else:
    y_train_pred[i]=0
cm = confusion_matrix(y_train, y_train_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("\nTraining Data Results: ")
print("Accuracy on training data:  ", round(model_lstm.evaluate(X_train, y_train)[1]*100,2), "%")
print("Precision on training data: ", round(precision*100,2), "%")
print("Recall on training data:    ", round(recall*100, 2), "%")
print("F1 score on training data:  ", round(f1*100, 2), "%")
print()
print("Test Data Results: ")
y_pred = model_lstm.predict(X_test)
for i in range(len(y_pred)):
  if (y_pred[i]>=0.5):
    y_pred[i]=1
  else:
    y_pred[i]=0
cm = confusion_matrix(y_test, y_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("Accuracy on test data:  ", round(model_lstm.evaluate(X_test, y_test)[1]*100,2), "%")
print("Precision on test data: ",  round(precision*100, 2), "%")
print("Recall on test data:    ", round(recall*100, 2), "%")
print("F1 score on test data:  ",  round(f1*100, 2), "%")

8956/8956 [==============================] - 583s 65ms/step

Training Data Results: 
8956/8956 [==============================] - 596s 67ms/step - loss: 0.2313 - accuracy: 0.9179
Accuracy on training data:   91.79 %
Precision on training data:  99.57 %
Recall on training data:     92.07 %
F1 score on training data:   95.67 %

Test Data Results: 
2239/2239 [==============================] - 151s 68ms/step - loss: 0.2304 - accuracy: 0.9184
Accuracy on test data:   91.84 %
Precision on test data:  99.57 %
Recall on test data:     92.11 %
F1 score on test data:   95.7 %


In [28]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Flatten
from tensorflow.keras.layers import Embedding, Bidirectional

model_bi_lstm = Sequential()

model_bi_lstm.add(LSTM(200, input_shape=(13, 1), dropout=0.2, return_sequences=True))
model_bi_lstm.add(Dense(100, activation="relu"))
model_bi_lstm.add(Bidirectional(LSTM(64, dropout=0.2)))
model_bi_lstm.add(Dense(5, activation="relu"))
model_bi_lstm.add(Flatten())
model_bi_lstm.add(Dense(1, activation="sigmoid"))


In [29]:
optimizer = Adam(learning_rate=3e-4)
model_bi_lstm.compile(loss="binary_crossentropy",
              optimizer=optimizer,
              metrics=["accuracy"])

In [30]:
history = model_bi_lstm.fit(X_train, y_train, epochs=2)

Epoch 1/2
8956/8956 [==============================] - 1623s 181ms/step - loss: 0.2354 - accuracy: 0.9173
Epoch 2/2
8956/8956 [==============================] - 1634s 182ms/step - loss: 0.2308 - accuracy: 0.9182


In [31]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_train_pred = model_bi_lstm.predict(X_train)
for i in range(len(y_train_pred)):
  if (y_train_pred[i]>=0.5):
    y_train_pred[i]=1
  else:
    y_train_pred[i]=0
cm = confusion_matrix(y_train, y_train_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("\nTraining Data Results: ")
print("Accuracy on training data:  ", round(model_bi_lstm.evaluate(X_train, y_train)[1]*100,2), "%")
print("Precision on training data: ", round(precision*100,2), "%")
print("Recall on training data:    ", round(recall*100, 2), "%")
print("F1 score on training data:  ", round(f1*100, 2), "%")
print()
print("Test Data Results: ")
y_pred = model_bi_lstm.predict(X_test)
for i in range(len(y_pred)):
  if (y_pred[i]>=0.5):
    y_pred[i]=1
  else:
    y_pred[i]=0
cm = confusion_matrix(y_test, y_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("Accuracy on test data:  ", round(model_bi_lstm.evaluate(X_test, y_test)[1]*100,2), "%")
print("Precision on test data: ",  round(precision*100, 2), "%")
print("Recall on test data:    ", round(recall*100, 2), "%")
print("F1 score on test data:  ",  round(f1*100, 2), "%")

8956/8956 [==============================] - 593s 66ms/step

Training Data Results: 
8956/8956 [==============================] - 619s 69ms/step - loss: 0.2267 - accuracy: 0.9198
Accuracy on training data:   91.98 %
Precision on training data:  99.17 %
Recall on training data:     92.55 %
F1 score on training data:   95.75 %

Test Data Results: 
2239/2239 [==============================] - 154s 69ms/step - loss: 0.2260 - accuracy: 0.9199
Accuracy on test data:   91.99 %
Precision on test data:  99.17 %
Recall on test data:     92.57 %
F1 score on test data:   95.76 %


In [7]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

In [33]:
#XGBOOST
from xgboost import XGBClassifier


classifier_xgb = XGBClassifier(max_depth= 3, learning_rate= .15, n_estimators= 300, nthread=-1, silent= True, subsample= 0.9)
classifier_xgb.fit(X_train, y_train)
accuracies = cross_val_score(classifier_xgb, X_train,  y_train, cv = 3)
y_train_pred = classifier_xgb.predict(X_train)
cm = confusion_matrix(y_train, y_train_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("\nTraining Data Results: ")
print("Accuracy on training data:  ", round((accuracies.mean()*100.0), 2), "%")
print("Precision on training data: ", round(precision*100,2), "%")
print("Recall on training data:    ", round(recall*100, 2), "%")
print("F1 score on training data:  ", round(f1*100, 2), "%")
print()
print("Test Data Results: ")
y_pred = classifier_xgb.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("Accuracy on test data:  ", round(accuracy_score(y_test, y_pred)*100, 2), "%")
print("Precision on test data: ",  round(precision*100, 2), "%")
print("Recall on test data:    ", round(recall*100, 2), "%")
print("F1 score on test data:  ",  round(f1*100, 2), "%")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:53:37] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:53:37] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:53:38] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [13:53:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)



Training Data Results: 
Accuracy on training data:   92.01 %
Precision on training data:  99.03 %
Recall on training data:     92.82 %
F1 score on training data:   95.82 %

Test Data Results: 
Accuracy on test data:   92.07 %
Precision on test data:  98.97 %
Recall on test data:     92.81 %
F1 score on test data:   95.79 %


In [34]:
#RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier

classifier_rf = RandomForestClassifier(n_estimators=100, max_depth= 50, random_state=30)
classifier_rf.fit(X_train, y_train)
accuracies = cross_val_score(classifier_rf, X_train,  y_train, cv = 3)
y_train_pred = classifier_rf.predict(X_train)
cm = confusion_matrix(y_train, y_train_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("\nTraining Data Results: ")
print("Accuracy on training data:  ", round((accuracies.mean()*100.0), 2), "%")
print("Precision on training data: ", round(precision*100,2), "%")
print("Recall on training data:    ", round(recall*100, 2), "%")
print("F1 score on training data:  ", round(f1*100, 2), "%")
print()
print("Test Data Results: ")
y_pred = classifier_rf.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("Accuracy on test data:  ", round(accuracy_score(y_test, y_pred)*100, 2), "%")
print("Precision on test data: ",  round(precision*100, 2), "%")
print("Recall on test data:    ", round(recall*100, 2), "%")
print("F1 score on test data:  ",  round(f1*100, 2), "%")


Training Data Results: 
Accuracy on training data:   91.89 %
Precision on training data:  100.0 %
Recall on training data:     100.0 %
F1 score on training data:   100.0 %

Test Data Results: 
Accuracy on test data:   91.9 %
Precision on test data:  98.8 %
Recall on test data:     92.78 %
F1 score on test data:   95.7 %


In [35]:
#DECISION TREE
from sklearn.tree import DecisionTreeClassifier
classifier_dt = DecisionTreeClassifier(criterion='entropy', max_depth=150)
classifier_dt.fit(X_train, y_train)
accuracies = cross_val_score(classifier_dt, X_train,  y_train, cv = 3)
y_train_pred = classifier_dt.predict(X_train)
cm = confusion_matrix(y_train, y_train_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("\nTraining Data Results: ")
print("Accuracy on training data:  ", round((accuracies.mean()*100.0), 2), "%")
print("Precision on training data: ", round(precision*100,2), "%")
print("Recall on training data:    ", round(recall*100, 2), "%")
print("F1 score on training data:  ", round(f1*100, 2), "%")
print()
print("Test Data Results: ")
y_pred = classifier_dt.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("Accuracy on test data:  ", round(accuracy_score(y_test, y_pred)*100, 2), "%")
print("Precision on test data: ",  round(precision*100, 2), "%")
print("Recall on test data:    ", round(recall*100, 2), "%")
print("F1 score on test data:  ",  round(f1*100, 2), "%")


Training Data Results: 
Accuracy on training data:   86.94 %
Precision on training data:  100.0 %
Recall on training data:     100.0 %
F1 score on training data:   100.0 %

Test Data Results: 
Accuracy on test data:   86.84 %
Precision on test data:  92.33 %
Recall on test data:     93.17 %
F1 score on test data:   92.75 %


In [36]:
#EXTRA TREE
from sklearn.ensemble import ExtraTreesClassifier

classifier_et = ExtraTreesClassifier(n_estimators=100, max_depth=150,random_state=30)
classifier_et.fit(X_train, y_train)
accuracies = cross_val_score(classifier_et, X_train,  y_train, cv = 3)
y_train_pred = classifier_et.predict(X_train)
cm = confusion_matrix(y_train, y_train_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("\nTraining Data Results: ")
print("Accuracy on training data:  ", round((accuracies.mean()*100.0), 2), "%")
print("Precision on training data: ", round(precision*100,2), "%")
print("Recall on training data:    ", round(recall*100, 2), "%")
print("F1 score on training data:  ", round(f1*100, 2), "%")
print()
print("Test Data Results: ")
y_pred = classifier_et.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("Accuracy on test data:  ", round(accuracy_score(y_test, y_pred)*100, 2), "%")
print("Precision on test data: ",  round(precision*100, 2), "%")
print("Recall on test data:    ", round(recall*100, 2), "%")
print("F1 score on test data:  ",  round(f1*100, 2), "%")


Training Data Results: 
Accuracy on training data:   91.83 %
Precision on training data:  100.0 %
Recall on training data:     100.0 %
F1 score on training data:   100.0 %

Test Data Results: 
Accuracy on test data:   91.88 %
Precision on test data:  98.77 %
Recall on test data:     92.78 %
F1 score on test data:   95.68 %


In [37]:
# Logistic Regression
classifier_lr = LogisticRegression(C=0.1,  solver="liblinear", max_iter=500)
classifier_lr.fit(X_train, y_train)
accuracies = cross_val_score(classifier_lr, X_train,  y_train, cv = 3)
y_train_pred = classifier_lr.predict(X_train)
cm = confusion_matrix(y_train, y_train_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("\nTraining Data Results: ")
print("Accuracy on training data:  ", round((accuracies.mean()*100.0), 2), "%")
print("Precision on training data: ", round(precision*100,2), "%")
print("Recall on training data:    ", round(recall*100, 2), "%")
print("F1 score on training data:  ", round(f1*100, 2), "%")
print()
print("Test Data Results: ")
y_pred = classifier_lr.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("Accuracy on test data:  ", round(accuracy_score(y_test, y_pred)*100, 2), "%")
print("Precision on test data: ",  round(precision*100, 2), "%")
print("Recall on test data:    ", round(recall*100, 2), "%")
print("F1 score on test data:  ",  round(f1*100, 2), "%")


Training Data Results: 
Accuracy on training data:   91.93 %
Precision on training data:  98.87 %
Recall on training data:     92.75 %
F1 score on training data:   95.71 %

Test Data Results: 
Accuracy on test data:   91.94 %
Precision on test data:  98.88 %
Recall on test data:     92.75 %
F1 score on test data:   95.72 %


In [11]:
params_sgd = {
    "loss" : ["hinge", "log", "huber", "modified_huber", "perceptron", "epsilon_insensitive"],
    "alpha" : [10**-i for i in range (1, 6)],
    "penalty" : ["l2", "elasticnet"],
    "class_weight" : [None],
    "tol": [0.001],
    "max_iter": [1000],
    "l1_ratio": [i/10 for i in range(1,10)]
}

model = SGDClassifier(random_state = 0, shuffle=True)
classifier_sgd = GridSearchCV(estimator = model,
                           param_grid = params_sgd,
                           scoring = 'roc_auc',
                           cv = 3,
                           n_jobs = -1,
                           return_train_score=True,
                           verbose=10)
classifier_sgd.fit(X_train, y_train)
# accuracies = cross_val_score(classifier_sgd, X_train,  y_train, cv = 3)
y_train_pred = classifier_sgd.predict(X_train)
cm = confusion_matrix(y_train, y_train_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("\nTraining Data Results: ")
print("Accuracy on training data:  ",  round(accuracy_score(y_train, y_train_pred)*100, 2), "%")
print("Precision on training data: ", round(precision*100,2), "%")
print("Recall on training data:    ", round(recall*100, 2), "%")
print("F1 score on training data:  ", round(f1*100, 2), "%")
print()
print("Test Data Results: ")
y_pred = classifier_sgd.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("Accuracy on test data:  ", round(accuracy_score(y_test, y_pred)*100, 2), "%")
print("Precision on test data: ",  round(precision*100, 2), "%")
print("Recall on test data:    ", round(recall*100, 2), "%")
print("F1 score on test data:  ",  round(f1*100, 2), "%")

Fitting 3 folds for each of 540 candidates, totalling 1620 fits

Training Data Results: 
Accuracy on training data:   91.77 %
Precision on training data:  99.62 %
Recall on training data:     92.02 %
F1 score on training data:   95.67 %

Test Data Results: 
Accuracy on test data:   91.84 %
Precision on test data:  99.63 %
Recall on test data:     92.06 %
F1 score on test data:   95.7 %


In [12]:
#Voting

params_sgd = {
    "loss" : ["hinge", "log", "huber", "modified_huber", "perceptron", "epsilon_insensitive"],
    "alpha" : [10**-i for i in range (1, 6)],
    "penalty" : ["l2", "elasticnet"],
    "class_weight" : [None],
    "tol": [0.001],
    "max_iter": [1000],
    "l1_ratio": [i/10 for i in range(1,10)]
}

sgd_model = SGDClassifier(random_state = 0, shuffle=True)

classifier1 = XGBClassifier(max_depth= 3, learning_rate= .15, n_estimators= 300, nthread=-1, silent= True, subsample= 0.9)
classifier2 = RandomForestClassifier(n_estimators=100, max_depth= 50, random_state=30)
classifier3 = DecisionTreeClassifier(criterion='entropy', max_depth=150)
classifier4 = ExtraTreesClassifier(n_estimators=100, max_depth=150,random_state=30)
classifier5 = LogisticRegression(C=0.1,  solver="liblinear", max_iter=500)
classifier6 = GridSearchCV(estimator = sgd_model, param_grid = params_sgd, scoring = 'roc_auc', cv = 3, n_jobs = -1, return_train_score=True, verbose=10)


classifier_vote= VotingClassifier(estimators=[('XGB', classifier1), ('RF', classifier2), ('DT', classifier3),('ET', classifier4),("LR", classifier5),
                                         ("SGD", classifier6)], voting='hard')
classifier_vote.fit(X_train, y_train)
# accuracies = cross_val_score(classifier_vote, X_train,  y_train, cv = 3)
y_train_pred = classifier_vote.predict(X_train)
cm = confusion_matrix(y_train, y_train_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("\nTraining Data Results: ")
print("Accuracy on training data:  ", round(accuracy_score(y_train, y_train_pred)*100, 2), "%")
print("Precision on training data: ", round(precision*100,2), "%")
print("Recall on training data:    ", round(recall*100, 2), "%")
print("F1 score on training data:  ", round(f1*100, 2), "%")
print()
print("Test Data Results: ")
y_pred = classifier_vote.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
precision = (cm[0,0])/(cm[0,0]+cm[0,1])
recall = (cm[0,0])/(cm[0,0]+cm[1,0])
f1= 2*((precision*recall)/(precision+recall))
print("Accuracy on test data:  ", round(accuracy_score(y_test, y_pred)*100, 2), "%")
print("Precision on test data: ",  round(precision*100, 2), "%")
print("Recall on test data:    ", round(recall*100, 2), "%")
print("F1 score on test data:  ",  round(f1*100, 2), "%")


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:160: UserWarning: [14:05:11] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "silent" } are not used.

  warnings.warn(smsg, UserWarning)


Fitting 3 folds for each of 540 candidates, totalling 1620 fits

Training Data Results: 
Accuracy on training data:   93.22 %
Precision on training data:  100.0 %
Recall on training data:     93.07 %
F1 score on training data:   96.41 %

Test Data Results: 
Accuracy on test data:   92.0 %
Precision on test data:  99.22 %
Recall on test data:     92.54 %
F1 score on test data:   95.76 %


In [13]:
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler


"""The best performing model on test dataset is XGB"""

# period 3 graph pre-processing
filename = "W_LinkPred_i3_NonExistingLinks.csv"
non_existing_links_i3 = pd.read_csv(filename)
filename = "W_LinkPred_i3_ExistingLinks.csv"
existing_links_i3 = pd.read_csv(filename)

non_existing_links_i3["i3"]= [0 for i in range(len(non_existing_links_i3))]
existing_links_i3["i3"]= [1 for i in range(len(existing_links_i3))]
data_i3 = pd.concat([non_existing_links_i3,existing_links_i3])
data_i3 = data_i3.sort_values(by=["u","v"])

u = data_i3['u']
v = data_i3['v']
i3 = data_i3['i3']

data_i3_uv = pd.DataFrame({'u': u, 'v': v, 'i3':i3})
data_i3_uv.to_csv('data_i3.csv')

X_predict = pd.DataFrame(data_i3.iloc[:,[4,5,6,7,8,9,10,11,12,13,14,15,16]].values)
data = X_predict.round(4)
X = data.iloc[:,[0,1,2,3,4,5,6,7,8,9,10,11,12]].values

sc = StandardScaler()
X_predict = sc.fit_transform(X)

warnings.simplefilter(action='ignore', category=FutureWarning)

# year 2017
df = pd.read_csv("co_occurrence_matrix_1.csv")
df3= pd.read_csv('data_i3.csv')
df = df.set_index("Unnamed: 0", drop = True)

df1 = pd.DataFrame(columns=['u', 'v', 'i1'])

for ii in range(len(df3)):
  i = df3["u"][ii]
  j = df3["v"][ii]
  if df.loc[i, j] != 0:
      new_data = {'u': i, 'v':j, 'i1': 1}
      df1 = df1.append(new_data, ignore_index=True)
  else:
      new_data = {'u': i, 'v':j, 'i1': 0}
      df1 = df1.append(new_data, ignore_index=True)
  # if(ii==10):
  #   break
df1.to_csv('data_i1.csv', index=False)
data_i1 = pd.read_csv('data_i1.csv')

# year 2018
df = pd.read_csv("co_occurrence_matrix_2.csv")
df3= pd.read_csv('data_i3.csv')
df = df.set_index("Unnamed: 0", drop = True)

df1 = pd.DataFrame(columns=['u', 'v', 'i2'])

for ii in range(len(df3)):
  i = df3["u"][ii]
  j = df3["v"][ii]
  if df.loc[i, j] != 0:
      new_data = {'u': i, 'v':j, 'i2': 1}
      df1 = df1.append(new_data, ignore_index=True)
  else:
      new_data = {'u': i, 'v':j, 'i2': 0}
      df1 = df1.append(new_data, ignore_index=True)
df1.to_csv('data_i2.csv', index=False)
data_i2 = pd.read_csv('data_i2.csv')



In [18]:
data_i1 = pd.read_csv('data_i1.csv')
data_i2 = pd.read_csv('data_i2.csv')
data_i3 = pd.read_csv('data_i3.csv')
final_table = pd.DataFrame()
final_table["u"] = data_i1["u"]
final_table["v"] = data_i1["v"]
final_table["2017"] = data_i1["i1"]
final_table["2018"] = data_i2["i2"]
final_table["2019"] = data_i3["i3"]

# forcasting bigrams for year 2020
model_pred = classifier_xgb.predict(X_predict)

final_table["2020"] = model_pred
final_table.to_csv('Bigrams Occurences of 4 years.csv')
print(final_table.head())

            u         v  2017  2018  2019  2020
0  acceptable    access     0     0     0     0
1  acceptable      acid     0     0     1     0
2  acceptable    action     0     0     0     0
3  acceptable    active     1     0     0     0
4  acceptable  activity     0     0     0     0


In [19]:
print("Length of final_table:", len(final_table))
print("Length of model_pred:", len(model_pred))


Length of final_table: 179700
Length of model_pred: 179700


In [20]:
final_table

,u,v,2017,2018,2019,2020
0,acceptable,access,0,0,0,0
1,acceptable,acid,0,0,1,0
2,acceptable,action,0,0,0,0
3,acceptable,active,1,0,0,0
4,acceptable,activity,0,0,0,0
...,...,...,...,...,...,...
179695,working,xray,0,0,0,0
179696,working,zone,0,0,0,0
179697,wound,xray,0,0,0,0
179698,wound,zone,0,0,0,0


In [21]:
import pandas as pd
# extracting future bigrams
condition = (final_table['2017'] == 0) & (final_table['2018'] == 0) & (final_table['2019'] == 0) & (final_table['2020'] == 1)
new_df = final_table[condition].copy()
new_df


,u,v,2017,2018,2019,2020
4379,adapted,fluid,0,0,0,1
6040,adjacent,comprising,0,0,0,1
6200,adjacent,includes,0,0,0,1
6201,adjacent,including,0,0,0,1
10876,along,includes,0,0,0,1
...,...,...,...,...,...,...
172724,second,support,0,0,0,1
172761,second,two,0,0,0,1
173503,sensor,source,0,0,0,1
175555,source,tissue,0,0,0,1


In [22]:
new_df = new_df.drop("2017", axis=1)
new_df = new_df.drop("2018", axis=1)
new_df = new_df.drop("2019", axis=1)
new_df = new_df.drop("2020", axis=1)
new_df

,u,v
4379,adapted,fluid
6040,adjacent,comprising
6200,adjacent,includes
6201,adjacent,including
10876,along,includes
...,...,...
172724,second,support
172761,second,two
173503,sensor,source
175555,source,tissue


In [23]:
future_bigrams_dict={'f_b':[]}
for i in range(len(new_df)):
  future_bigrams_dict['f_b'].append(new_df.iloc[i,0]+" " +new_df.iloc[i,1])
future_bigrams = pd.DataFrame(future_bigrams_dict)
future_bigrams.to_csv('Future_bigrams.csv', header=True, index = False)



In [24]:
pd.read_csv("Future_bigrams.csv")

,f_b
0,adapted fluid
1,adjacent comprising
2,adjacent includes
3,adjacent including
4,along includes
...,...
101,second support
102,second two
103,sensor source
104,source tissue


In [ ]:
# import pandas as pd
# import regex
# from nltk.util import skipgrams
# from time import time
# t0 = time()
# file_text_t3 = "t3.csv"

# promising_bigrams = set(pd.read_csv("Future_bigrams.csv", header = 0, index_col= 0).index)

# terms_len = []
# for i in promising_bigrams:
#     tpl = i.split()
#     if tpl[0] == tpl[1]:
#         print("duplicate")
#     for word in tpl:
#         terms_len.append(len(word))
# print(min(terms_len))
# print(max(terms_len))

# """Read csv file"""
# df = pd.read_csv(file_text_t3, header = 0).dropna()

# promising_patents = []
# number_bigrams_list = []
# number_unique_bigrams_list = []
# unique_bigrams_list = []
# number_possible_unique_bigrams = []
# unqiue_ratio_promising_to_possible = []
# counter = 1
# t1 = time()

# for patent, row in df.iterrows():
#     # patent_raw = patent
#     # patent = patent.split()[1]
#     # patent = patent.split("#")[1]
#     # patent = patent.split(",")
#     # patent = "".join(patent)
#     texts_raw = row["Title"] + " " + row["Abstract"] + " " + row["Claims"]
#     texts_raw = regex.sub(r'\P{L}+', ' ', texts_raw)
#     texts_raw.replace("."," ")
#     texts_raw.replace("€"," ")
#     texts_raw.replace("$"," ")
#     texts_raw.replace(","," ")
#     texts_raw.replace("!"," ")
#     texts_raw.replace("?"," ")
#     texts_raw.replace("&"," ")
#     texts_raw.replace("-"," ")
#     texts_raw.replace(";"," ")
#     texts_raw.replace("/"," ")
#     texts_raw.replace(")"," ")
#     texts_raw.replace("("," ")
#     texts_raw.replace("+"," ")
#     texts_raw.replace("="," ")
#     texts_raw.replace("\\"," ")
#     texts_raw.replace(":"," ")
#     texts_raw.replace("'"," ")
#     texts_raw.replace("`"," ")
#     texts_raw.replace("´"," ")
#     texts_raw.replace("#"," ")
#     texts_raw.replace("%"," ")
#     texts_raw.replace("§"," ")
#     texts_raw.replace("ß"," ")

#     #tokenize and lowercase
#     texts_raw = texts_raw.lower().split()
#     texts_raw = [word for word in texts_raw if len(word) > 3]
#     texts_raw = set(texts_raw)
#     bigrams_raw = set(skipgrams(texts_raw,2,len(texts_raw)*2))
#     bigrams = []
#     for word in bigrams_raw:
#         l = [word[0], word[1]]
#         l.sort()
#         bigrams.append(str(l[0]) + " " + str(l[1]))

#     number_unique_bigrams = 0
#     unique_bigrams = []

#     bigrams = set(bigrams)
#     for promising_bigram in promising_bigrams:
#         if promising_bigram in bigrams:
#             number_unique_bigrams += 1
#             unique_bigrams.append(promising_bigram)

#     unique_bigrams.sort()
#     unique_bigrams_2 = ""
#     for bigram in unique_bigrams:
#         if len(unique_bigrams_2) == 0:
#             unique_bigrams_2 += bigram
#         else:
#             unique_bigrams_2 += ", "
#             unique_bigrams_2 += bigram
#     promising_patents.append(patent)
#     number_possible_unique_bigrams.append(len(bigrams))
#     number_unique_bigrams_list.append(number_unique_bigrams)
#     unique_bigrams_list.append(unique_bigrams_2)
#     unqiue_ratio_promising_to_possible.append(round(number_unique_bigrams/len(bigrams),4))
#     if counter%500==0:
#         print("Next 500 patents processed in {0} seconds ({1} of {2}).".format(int(time()-t1), counter, len(df)))
#         t1 = time()
#     counter += 1

# output = pd.DataFrame()
# output["promising patents"] = promising_patents
# output["# unique bigrams"] = number_unique_bigrams_list
# output["# possible unique bigrams"] = number_possible_unique_bigrams
# output["ratio"] = unqiue_ratio_promising_to_possible
# output["unique bigrams"] = unique_bigrams_list
# output.to_excel("Promisingness_score_t3_sorted.xlsx", index=False)
# print("\nFinished in {0} seconds.".format(int(time()-t0)))

3
13
Next 500 patents processed in 14 seconds (500 of 596).

Finished in 19 seconds.
